<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_Spark_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting OpenAI with Apache Spark

Introduction of [pyspark-ai](https://github.com/databrickslabs/pyspark-ai)

Pyspark-AI takes English instructions and compile them into PySpark objects like DataFrames, to make Spark more user-friendly and accessible, allowing you to focus on extracting insights from your data.

In [1]:
!pip install --quiet --upgrade langchain openai pyspark-ai pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.7 MB/s eta 0:00:00


In [42]:
import os
os.environ['OPENAI_API_KEY'] = 'your openai api key'

1. Initialize the Spark AI instance

In [43]:
from langchain.chat_models import ChatOpenAI
from pyspark_ai import SparkAI

# If 'gpt-4' is unavailable, use 'gpt-3.5-turbo' (might lower output quality)
llm = ChatOpenAI(model_name='gpt-4', temperature=0)

spark_ai = SparkAI(llm=llm, verbose=True)

# Activate partial functions for Spark DataFrame
spark_ai.activate()

2. Create a dataframe via a HTTP URL

In this case, we are fetching the share holders of Apple, one of the best performing stock in US market.

In [ ]:
holders_dataframe = spark_ai.create_df("https://finance.yahoo.com/quote/AAPL/holders?p=AAPL")

INFO: Parsing URL: https://finance.yahoo.com/quote/AAPL/holders?p=AAPL



INFO:spark_ai:Parsing URL: https://finance.yahoo.com/quote/AAPL/holders?p=AAPL



INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW apple_stock_holders AS 
SELECT * FROM VALUES
('Vanguard Group, Inc. (The)', 1309785362, 'Mar 30, 2023', 8.33, 254059068265),
('Blackrock Inc.', 1035008939, 'Mar 30, 2023', 6.58, 200760685161),
('Berkshire Hathaway, Inc', 915560382, 'Mar 30, 2023', 5.82, 177591248414),
('State Street Corporation', 576281774, 'Mar 30, 2023', 3.66, 111781376406),
('FMR, LLC', 311437576, 'Mar 30, 2023', 1.98, 60409546996),
('Geode Capital Management, LLC', 285171112, 'Mar 30, 2023', 1.81, 55314640942),
('Price (T.Rowe) Associates Inc', 234017381, 'Mar 30, 2023', 1.49, 45392351678),
('Morgan Stanley', 200615893, 'Mar 30, 2023', 1.28, 38913465010),
('Northern Trust Corporation', 173130542, 'Mar 30, 2023', 1.10, 33582131443),
('Norges Bank Investment Management', 167374278, 'Dec 30, 2022', 1.06, 32465588907)
AS v1(holder_name, shares, date_reported, percent_out, value)



INFO:spark_ai:SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW apple_stock_holders AS 
SELECT * FROM VALUES
('Vanguard Group, Inc. (The)', 1309785362, 'Mar 30, 2023', 8.33, 254059068265),
('Blackrock Inc.', 1035008939, 'Mar 30, 2023', 6.58, 200760685161),
('Berkshire Hathaway, Inc', 915560382, 'Mar 30, 2023', 5.82, 177591248414),
('State Street Corporation', 576281774, 'Mar 30, 2023', 3.66, 111781376406),
('FMR, LLC', 311437576, 'Mar 30, 2023', 1.98, 60409546996),
('Geode Capital Management, LLC', 285171112, 'Mar 30, 2023', 1.81, 55314640942),
('Price (T.Rowe) Associates Inc', 234017381, 'Mar 30, 2023', 1.49, 45392351678),
('Morgan Stanley', 200615893, 'Mar 30, 2023', 1.28, 38913465010),
('Northern Trust Corporation', 173130542, 'Mar 30, 2023', 1.10, 33582131443),
('Norges Bank Investment Management', 167374278, 'Dec 30, 2022', 1.06, 32465588907)
AS v1(holder_name, shares, date_reported, percent_out, value)



INFO: Storing data into temp view: apple_stock_holders



INFO:spark_ai:Storing data into temp view: apple_stock_holders



In [ ]:
holders_dataframe.show(n=5)

+--------------------+----------+-------------+-----------+------------+
|         holder_name|    shares|date_reported|percent_out|       value|
+--------------------+----------+-------------+-----------+------------+
|Vanguard Group, I...|1309785362| Mar 30, 2023|       8.33|254059068265|
|      Blackrock Inc.|1035008939| Mar 30, 2023|       6.58|200760685161|
|Berkshire Hathawa...| 915560382| Mar 30, 2023|       5.82|177591248414|
|State Street Corp...| 576281774| Mar 30, 2023|       3.66|111781376406|
|            FMR, LLC| 311437576| Mar 30, 2023|       1.98| 60409546996|
+--------------------+----------+-------------+-----------+------------+
only showing top 5 rows



3. Plot

In [ ]:
holders_dataframe.ai.plot()

INFO: Here is a Python code snippet that uses Plotly to visualize the data in the PySpark DataFrame `df`. This code assumes that you want to create a bar chart with `holder_name` on the x-axis and `shares` on the y-axis. 


```
import plotly.graph_objects as go
from pyspark.sql.functions import col

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = df.select("*").toPandas()

# Create a bar chart
fig = go.Figure(data=go.Bar(x=pandas_df['holder_name'], y=pandas_df['shares']))

# Add title and labels
fig.update_layout(title='Apple Stock Holders', xaxis_title='Holder Name', yaxis_title='Shares')

# Display the plot
fig.show()
```

This code first converts the PySpark DataFrame to a Pandas DataFrame using the `toPandas()` method. Then, it creates a bar chart using Plotly's `go.Bar` function, with `holder_name` as the x-axis and `shares` as the y-axis. The `update_layout` method is used to add a title and labels to the plot. Finally, the `show` method is used to display the plo

INFO:spark_ai:Here is a Python code snippet that uses Plotly to visualize the data in the PySpark DataFrame `df`. This code assumes that you want to create a bar chart with `holder_name` on the x-axis and `shares` on the y-axis. 


```
import plotly.graph_objects as go
from pyspark.sql.functions import col

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = df.select("*").toPandas()

# Create a bar chart
fig = go.Figure(data=go.Bar(x=pandas_df['holder_name'], y=pandas_df['shares']))

# Add title and labels
fig.update_layout(title='Apple Stock Holders', xaxis_title='Holder Name', yaxis_title='Shares')

# Display the plot
fig.show()
```

This code first converts the PySpark DataFrame to a Pandas DataFrame using the `toPandas()` method. Then, it creates a bar chart using Plotly's `go.Bar` function, with `holder_name` as the x-axis and `shares` as the y-axis. The `update_layout` method is used to add a title and labels to the plot. Finally, the `show` method is used to display

In [ ]:
holders_dataframe.ai.plot("Pie chart for Apple's top holders, show their name and share percentages")

INFO: Here is a Python code snippet that uses PySpark and Plotly to visualize the result of `df` as a pie chart. This code assumes that the `percent_out` column represents the share percentages of each holder.


```
import plotly.graph_objects as go
from pyspark.sql import SparkSession

# Assuming that SparkSession is already initialized
spark = SparkSession.builder.getOrCreate()

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = df.toPandas()

# Create a pie chart with Plotly
fig = go.Figure(data=[go.Pie(labels=pandas_df['holder_name'], values=pandas_df['percent_out'], hole=.3)])

# Set the title of the chart
fig.update_layout(title_text="Apple's Top Holders Share Percentages")

# Display the chart
fig.show()
```

This code first converts the Spark DataFrame `df` to a Pandas DataFrame using the `toPandas()` method. Then, it creates a pie chart using Plotly's `go.Pie` function, with the holder names as labels and the share percentages as values. The `hole` parameter is se

INFO:spark_ai:Here is a Python code snippet that uses PySpark and Plotly to visualize the result of `df` as a pie chart. This code assumes that the `percent_out` column represents the share percentages of each holder.


```
import plotly.graph_objects as go
from pyspark.sql import SparkSession

# Assuming that SparkSession is already initialized
spark = SparkSession.builder.getOrCreate()

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = df.toPandas()

# Create a pie chart with Plotly
fig = go.Figure(data=[go.Pie(labels=pandas_df['holder_name'], values=pandas_df['percent_out'], hole=.3)])

# Set the title of the chart
fig.update_layout(title_text="Apple's Top Holders Share Percentages")

# Display the chart
fig.show()
```

This code first converts the Spark DataFrame `df` to a Pandas DataFrame using the `toPandas()` method. Then, it creates a pie chart using Plotly's `go.Pie` function, with the holder names as labels and the share percentages as values. The `hole` paramet

In [ ]:
top_holder_dataframe = holders_dataframe.ai.transform("name with the highest percentage, and its percentage")
top_holder_dataframe.show()

INFO: SQL query for the transform:
SELECT holder_name, MAX(percent_out) as max_percent
FROM temp_view_for_transform
GROUP BY holder_name
ORDER BY max_percent DESC
LIMIT 1



INFO:spark_ai:SQL query for the transform:
SELECT holder_name, MAX(percent_out) as max_percent
FROM temp_view_for_transform
GROUP BY holder_name
ORDER BY max_percent DESC
LIMIT 1



+--------------------+-----------+
|         holder_name|max_percent|
+--------------------+-----------+
|Vanguard Group, I...|       8.33|
+--------------------+-----------+



4. Explain what the AI did

In [ ]:
top_holder_dataframe.ai.explain()

'In summary, this dataframe is retrieving the holder name with the highest percentage of Apple stocks out of all holders. It presents the results sorted by the percentage of stocks in descending order and limits the result to the top holder.'

5. Verify the dataframe attributes by giving the expectatoin in natural language

In [ ]:
holders_dataframe.ai.verify("expect Apple's top holders have no more than 10% of shares")

INFO: Generated code:
def check_apple_top_holders(df) -> bool:
    from pyspark.sql.functions import col

    # Filter the DataFrame for rows where the holder_name is 'Apple'
    apple_df = df.filter(col('holder_name') == 'Apple')

    # Check if any row in the filtered DataFrame has a percent_out greater than 10
    if apple_df.filter(col('percent_out') > 10).count() > 0:
        return False
    else:
        return True

result = check_apple_top_holders(df)



INFO:spark_ai:Generated code:
def check_apple_top_holders(df) -> bool:
    from pyspark.sql.functions import col

    # Filter the DataFrame for rows where the holder_name is 'Apple'
    apple_df = df.filter(col('holder_name') == 'Apple')

    # Check if any row in the filtered DataFrame has a percent_out greater than 10
    if apple_df.filter(col('percent_out') > 10).count() > 0:
        return False
    else:
        return True

result = check_apple_top_holders(df)



INFO: 
Result: True


INFO:spark_ai:
Result: True


## More Exciting Use Cases

Let's see more use cases in which we will explore how we can use natual language in data processing and analysis more elegantly with pyspark-ai.

In [57]:
df = spark_ai.create_df("https://coinmarketcap.com/", ['name', 'price', 'market_cap', 'circulating_supply'])

INFO: Parsing URL: https://coinmarketcap.com/



INFO:spark_ai:Parsing URL: https://coinmarketcap.com/



INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW cryptocurrencies AS 
SELECT * FROM VALUES
('Bitcoin', '$31,050.33', '$602.79B', '19,419,081 BTC'),
('Ethereum', '$1,957.54', '$235.29B', '120,219,234 ETH'),
('Tether', '$1.00', '$83.35B', '83,341,708,027 USDT'),
('BNB', '$247.17', '$38.52B', '155,850,829 BNB'),
('USD Coin', '$1.00', '$27.66B', '27,653,660,244 USDC'),
('XRP', '$0.4886', '$25.53B', '52,254,289,650 XRP'),
('Cardano', '$0.2963', '$10.36B', '34,953,231,640 ADA'),
('Dogecoin', '$0.06809', '$9.53B', '140,024,896,384 DOGE'),
('Litecoin', '$106.61', '$7.81B', '73,282,214 LTC'),
('Solana', '$19.07', '$7.64B', '400,892,982 SOL')
AS v1(name, price, market_cap, circulating_supply)



INFO:spark_ai:SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW cryptocurrencies AS 
SELECT * FROM VALUES
('Bitcoin', '$31,050.33', '$602.79B', '19,419,081 BTC'),
('Ethereum', '$1,957.54', '$235.29B', '120,219,234 ETH'),
('Tether', '$1.00', '$83.35B', '83,341,708,027 USDT'),
('BNB', '$247.17', '$38.52B', '155,850,829 BNB'),
('USD Coin', '$1.00', '$27.66B', '27,653,660,244 USDC'),
('XRP', '$0.4886', '$25.53B', '52,254,289,650 XRP'),
('Cardano', '$0.2963', '$10.36B', '34,953,231,640 ADA'),
('Dogecoin', '$0.06809', '$9.53B', '140,024,896,384 DOGE'),
('Litecoin', '$106.61', '$7.81B', '73,282,214 LTC'),
('Solana', '$19.07', '$7.64B', '400,892,982 SOL')
AS v1(name, price, market_cap, circulating_supply)



INFO: Storing data into temp view: cryptocurrencies



INFO:spark_ai:Storing data into temp view: cryptocurrencies



In [58]:
df.show(n=5)

+--------+----------+----------+-------------------+
|    name|     price|market_cap| circulating_supply|
+--------+----------+----------+-------------------+
| Bitcoin|$31,050.33|  $602.79B|     19,419,081 BTC|
|Ethereum| $1,957.54|  $235.29B|    120,219,234 ETH|
|  Tether|     $1.00|   $83.35B|83,341,708,027 USDT|
|     BNB|   $247.17|   $38.52B|    155,850,829 BNB|
|USD Coin|     $1.00|   $27.66B|27,653,660,244 USDC|
+--------+----------+----------+-------------------+
only showing top 5 rows



In [59]:
df.ai.plot('Bar chart with name and price')

INFO: Here is a Python code snippet that uses Plotly to visualize the result of `df` as a bar chart with 'name' and 'price'. This code assumes that the 'price' column contains numerical values stored as strings.


```
import plotly.express as px
import pandas as pd

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Convert price column to numeric
pandas_df['price'] = pd.to_numeric(pandas_df['price'])

# Create a bar chart with 'name' and 'price'
fig = px.bar(pandas_df, x='name', y='price')

# Display the plot
fig.show()
```

This code first converts the Spark DataFrame to a Pandas DataFrame using the `toPandas()` method. Then, it converts the 'price' column to numeric using the `pd.to_numeric()` function. This is necessary because Plotly requires numerical data for the y-axis of a bar chart. Finally, it creates a bar chart with 'name' on the x-axis and 'price' on the y-axis using the `px.bar()` function, and displays the plot using the `show()` method.


INFO:spark_ai:Here is a Python code snippet that uses Plotly to visualize the result of `df` as a bar chart with 'name' and 'price'. This code assumes that the 'price' column contains numerical values stored as strings.


```
import plotly.express as px
import pandas as pd

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Convert price column to numeric
pandas_df['price'] = pd.to_numeric(pandas_df['price'])

# Create a bar chart with 'name' and 'price'
fig = px.bar(pandas_df, x='name', y='price')

# Display the plot
fig.show()
```

This code first converts the Spark DataFrame to a Pandas DataFrame using the `toPandas()` method. Then, it converts the 'price' column to numeric using the `pd.to_numeric()` function. This is necessary because Plotly requires numerical data for the y-axis of a bar chart. Finally, it creates a bar chart with 'name' on the x-axis and 'price' on the y-axis using the `px.bar()` function, and displays the plot using the `show()` method.


ValueError: ignored

In [60]:
transformed_df = df.ai.transform(
    """
    The price column is of string in the US currency format with comma separators, and denoted by US dollar sign.
    You must process correctly with such format, and add a custom column 'price_float' that transforms the price column to float type
    """
)

INFO: SQL query for the transform:
SELECT 
    *,
    CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS FLOAT) AS price_float
FROM 
    temp_view_for_transform



INFO:spark_ai:SQL query for the transform:
SELECT 
    *,
    CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS FLOAT) AS price_float
FROM 
    temp_view_for_transform



In [61]:
transformed_df.show(n=5)

+--------+----------+----------+-------------------+-----------+
|    name|     price|market_cap| circulating_supply|price_float|
+--------+----------+----------+-------------------+-----------+
| Bitcoin|$31,050.33|  $602.79B|     19,419,081 BTC|   31050.33|
|Ethereum| $1,957.54|  $235.29B|    120,219,234 ETH|    1957.54|
|  Tether|     $1.00|   $83.35B|83,341,708,027 USDT|        1.0|
|     BNB|   $247.17|   $38.52B|    155,850,829 BNB|     247.17|
|USD Coin|     $1.00|   $27.66B|27,653,660,244 USDC|        1.0|
+--------+----------+----------+-------------------+-----------+
only showing top 5 rows



In [62]:
transformed_df.ai.plot('Bar chart with name and float type price')

INFO: Here is the Python code to visualize the result of `df` using plotly. This code assumes that you have already created the `df` dataframe.


```
import plotly.express as px
import pandas as pd

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Create a bar chart with name and float type price
fig = px.bar(pandas_df, x='name', y='price_float', labels={'x':'Cryptocurrency', 'y':'Price'})

# Display the plot
fig.show()
```

This code first converts the Spark DataFrame to a Pandas DataFrame. Then, it uses the `plotly.express` module to create a bar chart with the cryptocurrency names on the x-axis and the float type prices on the y-axis. Finally, it displays the plot.


INFO:spark_ai:Here is the Python code to visualize the result of `df` using plotly. This code assumes that you have already created the `df` dataframe.


```
import plotly.express as px
import pandas as pd

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Create a bar chart with name and float type price
fig = px.bar(pandas_df, x='name', y='price_float', labels={'x':'Cryptocurrency', 'y':'Price'})

# Display the plot
fig.show()
```

This code first converts the Spark DataFrame to a Pandas DataFrame. Then, it uses the `plotly.express` module to create a bar chart with the cryptocurrency names on the x-axis and the float type prices on the y-axis. Finally, it displays the plot.


In [63]:
transformed_df = transformed_df.ai.transform(
    """
    The circulating_supply column is of string with comma separators, and trailing coin name symbol.
    Add a custom column 'circulating_supply_long' that transforms the circulating_supply column to long type
    """
)

INFO: SQL query for the transform:
SELECT 
    *,
    CAST(REPLACE(SUBSTRING_INDEX(circulating_supply, ' ', 1), ',', '') AS LONG) AS circulating_supply_long
FROM 
    temp_view_for_transform



INFO:spark_ai:SQL query for the transform:
SELECT 
    *,
    CAST(REPLACE(SUBSTRING_INDEX(circulating_supply, ' ', 1), ',', '') AS LONG) AS circulating_supply_long
FROM 
    temp_view_for_transform



In [64]:
transformed_df.show(n=5)

+--------+----------+----------+-------------------+-----------+-----------------------+
|    name|     price|market_cap| circulating_supply|price_float|circulating_supply_long|
+--------+----------+----------+-------------------+-----------+-----------------------+
| Bitcoin|$31,050.33|  $602.79B|     19,419,081 BTC|   31050.33|               19419081|
|Ethereum| $1,957.54|  $235.29B|    120,219,234 ETH|    1957.54|              120219234|
|  Tether|     $1.00|   $83.35B|83,341,708,027 USDT|        1.0|            83341708027|
|     BNB|   $247.17|   $38.52B|    155,850,829 BNB|     247.17|              155850829|
|USD Coin|     $1.00|   $27.66B|27,653,660,244 USDC|        1.0|            27653660244|
+--------+----------+----------+-------------------+-----------+-----------------------+
only showing top 5 rows



In [65]:
sorted_df = transformed_df.ai.transform('Sort by circulating_supply_long in a desc order')

INFO: SQL query for the transform:
SELECT * FROM temp_view_for_transform ORDER BY circulating_supply_long DESC



INFO:spark_ai:SQL query for the transform:
SELECT * FROM temp_view_for_transform ORDER BY circulating_supply_long DESC



In [66]:
sorted_df.show(n = 5)

+--------+--------+----------+--------------------+-----------+-----------------------+
|    name|   price|market_cap|  circulating_supply|price_float|circulating_supply_long|
+--------+--------+----------+--------------------+-----------+-----------------------+
|Dogecoin|$0.06809|    $9.53B|140,024,896,384 DOGE|    0.06809|           140024896384|
|  Tether|   $1.00|   $83.35B| 83,341,708,027 USDT|        1.0|            83341708027|
|     XRP| $0.4886|   $25.53B|  52,254,289,650 XRP|     0.4886|            52254289650|
| Cardano| $0.2963|   $10.36B|  34,953,231,640 ADA|     0.2963|            34953231640|
|USD Coin|   $1.00|   $27.66B| 27,653,660,244 USDC|        1.0|            27653660244|
+--------+--------+----------+--------------------+-----------+-----------------------+
only showing top 5 rows



In [67]:
sorted_df.ai.plot('Bar chart with name and long type circulating supply value')

INFO: Here is the Python code to visualize the result of `df` using plotly. This code will create a bar chart with the name of the cryptocurrency on the x-axis and the long type circulating supply value on the y-axis.


```
import plotly.express as px
import plotly.io as pio

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Create a bar chart
fig = px.bar(pandas_df, x='name', y='circulating_supply_long')

# Display the plot
pio.show(fig)
```

This code first converts the Spark DataFrame to a Pandas DataFrame using the `toPandas()` method. Then, it uses the `bar` function from the `plotly.express` module to create a bar chart. The x-axis is set to the 'name' column and the y-axis is set to the 'circulating_supply_long' column. Finally, the plot is displayed using the `show` function from the `plotly.io` module.


INFO:spark_ai:Here is the Python code to visualize the result of `df` using plotly. This code will create a bar chart with the name of the cryptocurrency on the x-axis and the long type circulating supply value on the y-axis.


```
import plotly.express as px
import plotly.io as pio

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Create a bar chart
fig = px.bar(pandas_df, x='name', y='circulating_supply_long')

# Display the plot
pio.show(fig)
```

This code first converts the Spark DataFrame to a Pandas DataFrame using the `toPandas()` method. Then, it uses the `bar` function from the `plotly.express` module to create a bar chart. The x-axis is set to the 'name' column and the y-axis is set to the 'circulating_supply_long' column. Finally, the plot is displayed using the `show` function from the `plotly.io` module.
